# Piecewise Linearity in Neural Networks: Interactive Demonstration

This notebook demonstrates one of the most fundamental properties of modern deep learning models: **piecewise linearity**.

## Key Insight

Neural networks with ReLU-like activation functions produce functions that are composed of multiple linear pieces, stitched together at boundaries where neurons switch from active to inactive.

---

## Setup

First, let's import the necessary libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib for better looking plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✓ Libraries imported successfully!")

---

## Part 1: Simple 1D Example

Let's start with the simplest possible example: a 1D neural network with just 3 hidden neurons.

### Network Architecture
- **Input**: 1D (a single number)
- **Hidden layer**: 3 neurons with ReLU activation
- **Output**: 1D (a single number)

Each neuron in the hidden layer activates at a different threshold, creating distinct linear regions.

In [ ]:
def tiny_network(x):
    """
    A simple 1D network: input -> 3 ReLU neurons -> output
    
    Hidden layer neurons:
      h1 = max(0, x + 1)  # Activates when x > -1
      h2 = max(0, x)       # Activates when x > 0
      h3 = max(0, x - 1)   # Activates when x > 1
    
    Output: 0.5*h1 - 1.0*h2 + 0.5*h3
    """
    h1 = np.maximum(0, x + 1)
    h2 = np.maximum(0, x)
    h3 = np.maximum(0, x - 1)
    
    output = 0.5 * h1 - 1.0 * h2 + 0.5 * h3
    return output

# Test the network
test_input = 0.5
test_output = tiny_network(test_input)
print(f"Network input: {test_input}")
print(f"Network output: {test_output:.4f}")
print("\n✓ Network defined successfully!")

### Visualizing the Piecewise Linear Function

Now let's visualize this function and see the piecewise linear structure clearly:

In [ ]:
# Generate data points
x = np.linspace(-2, 2, 1000)
y = tiny_network(x)

# Create the plot
fig, ax = plt.subplots(figsize=(14, 6))

# Plot the function
ax.plot(x, y, linewidth=3, color='darkblue', label='Network Output')

# Mark the activation boundaries
ax.axvline(-1, color='red', linestyle='--', alpha=0.7, linewidth=2, label='Neuron 1 activates (x = -1)')
ax.axvline(0, color='green', linestyle='--', alpha=0.7, linewidth=2, label='Neuron 2 activates (x = 0)')
ax.axvline(1, color='purple', linestyle='--', alpha=0.7, linewidth=2, label='Neuron 3 activates (x = 1)')

# Highlight the "kinks" where the function changes slope
kink_points = [-1, 0, 1]
kink_values = [tiny_network(np.array([k]))[0] for k in kink_points]
ax.scatter(kink_points, kink_values, color='red', s=100, zorder=5, label='Kink points')

# Labels and styling
ax.set_xlabel('Input (x)', fontsize=14, fontweight='bold')
ax.set_ylabel('Output', fontsize=14, fontweight='bold')
ax.set_title('Piecewise Linear Function from ReLU Network\n(Notice the kinks at activation boundaries)', 
             fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(loc='best', fontsize=11)

plt.tight_layout()
plt.show()

print("Notice how the function is perfectly linear between the dashed lines!")
print("Each section has a constant slope - that's what 'piecewise linear' means.")

### Analyzing Each Linear Region

Let's break down what's happening in each region:

In [ ]:
print("="*70)
print("LINEAR REGIONS ANALYSIS")
print("="*70)
print("\nFor network: output = 0.5*max(0,x+1) - 1.0*max(0,x) + 0.5*max(0,x-1)\n")

regions = [
    ("Region 1: x < -1", "All neurons OFF", "output = 0", 0),
    ("Region 2: -1 ≤ x < 0", "Only neuron 1 ON", "output = 0.5(x+1) = 0.5x + 0.5", 0.5),
    ("Region 3: 0 ≤ x < 1", "Neurons 1,2 ON", "output = 0.5(x+1) - 1.0x = -0.5x + 0.5", -0.5),
    ("Region 4: x ≥ 1", "All neurons ON", "output = 0.5(x+1) - 1.0x + 0.5(x-1) = 0", 0)
]

for region_name, neurons, formula, slope in regions:
    print(f"{region_name}")
    print(f"  Activation: {neurons}")
    print(f"  Formula: {formula}")
    print(f"  Slope: {slope}")
    print()

print("Key insight: Within each region, the function is EXACTLY LINEAR.")
print("The overall function is made up of these linear pieces!")

### Verification: Computing Derivatives

If the function is truly piecewise linear, the first derivative (slope) should be constant within each region, and the second derivative should be zero (except at the boundaries).

In [ ]:
# Compute numerical derivatives
dy_dx = np.gradient(y, x)  # First derivative (slope)
d2y_dx2 = np.gradient(dy_dx, x)  # Second derivative (curvature)

# Create subplots
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: First derivative (slope)
ax1 = axes[0]
ax1.plot(x, dy_dx, linewidth=2.5, color='darkred', label='First Derivative (slope)')
ax1.axvline(-1, color='red', linestyle='--', alpha=0.5)
ax1.axvline(0, color='green', linestyle='--', alpha=0.5)
ax1.axvline(1, color='purple', linestyle='--', alpha=0.5)
ax1.axhline(0, color='black', linestyle='-', alpha=0.2)
ax1.set_xlabel('Input (x)', fontsize=12)
ax1.set_ylabel('dy/dx', fontsize=12)
ax1.set_title('First Derivative: Constant Within Each Region\n(This proves the function is piecewise linear!)', 
              fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11)

# Plot 2: Second derivative (curvature)
ax2 = axes[1]
ax2.plot(x, d2y_dx2, linewidth=2.5, color='darkgreen', label='Second Derivative (curvature)')
ax2.axvline(-1, color='red', linestyle='--', alpha=0.5, label='Activation boundaries')
ax2.axvline(0, color='green', linestyle='--', alpha=0.5)
ax2.axvline(1, color='purple', linestyle='--', alpha=0.5)
ax2.axhline(0, color='black', linestyle='-', alpha=0.2)
ax2.set_xlabel('Input (x)', fontsize=12)
ax2.set_ylabel('d²y/dx²', fontsize=12)
ax2.set_title('Second Derivative: Zero Except at Boundaries\n(Zero curvature = linear function)', 
              fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=11)
ax2.set_ylim(-0.5, 0.5)

plt.tight_layout()
plt.show()

print("✓ The second derivative is essentially zero within each region!")
print("✓ This mathematically proves the function is piecewise linear.")

---

## Part 2: 2D Neural Network Example

Now let's look at a more realistic example with 2D inputs. This creates a piecewise linear **surface** instead of a curve.

### Network Architecture
- **Input**: 2D (two numbers)
- **Hidden layer**: 4 neurons with ReLU activation
- **Output**: 1D (a single number)

In [ ]:
def neural_network_2d(X, W1, b1, W2, b2):
    """
    2-layer network: input(2D) -> hidden(4 neurons, ReLU) -> output(1D)
    
    Args:
        X: Input array of shape (n_samples, 2)
        W1: First layer weights, shape (2, 4)
        b1: First layer biases, shape (4,)
        W2: Second layer weights, shape (4, 1)
        b2: Second layer bias, shape (1,)
    
    Returns:
        Output array of shape (n_samples, 1)
    """
    # First layer with ReLU activation
    hidden = np.maximum(0, X @ W1 + b1)
    
    # Output layer (linear)
    output = hidden @ W2 + b2
    
    return output

# Initialize random weights
np.random.seed(42)
W1 = np.random.randn(2, 4) * 0.5
b1 = np.random.randn(4) * 0.5
W2 = np.random.randn(4, 1) * 0.5
b2 = np.random.randn(1) * 0.5

print("✓ 2D Neural network initialized!")
print(f"\nNetwork architecture:")
print(f"  Input dimension: 2")
print(f"  Hidden layer: 4 neurons with ReLU")
print(f"  Output dimension: 1")
print(f"\nMaximum possible linear regions: 2^4 = {2**4}")

### Creating the Piecewise Linear Surface

Let's generate a grid of input points and compute the network output for each point:

In [ ]:
# Create a grid of input points
x1_range = np.linspace(-2, 2, 150)
x2_range = np.linspace(-2, 2, 150)
X1, X2 = np.meshgrid(x1_range, x2_range)

# Flatten for computation
X_flat = np.column_stack([X1.ravel(), X2.ravel()])

# Compute network output for all points
Y_flat = neural_network_2d(X_flat, W1, b1, W2, b2)
Y = Y_flat.reshape(X1.shape)

print(f"✓ Computed network output for {len(X_flat):,} points")
print(f"  Grid size: {X1.shape[0]} x {X1.shape[1]}")
print(f"  Output range: [{Y.min():.3f}, {Y.max():.3f}]")

### 3D Visualization: The Piecewise Linear Surface

In [ ]:
# Create 3D surface plot
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(X1, X2, Y, cmap=cm.viridis, 
                       alpha=0.9, linewidth=0, antialiased=True,
                       edgecolor='none')

# Add contour lines at the base
ax.contour(X1, X2, Y, zdir='z', offset=Y.min()-0.5, cmap=cm.viridis, alpha=0.5)

# Labels and title
ax.set_xlabel('Input x₁', fontsize=12, fontweight='bold')
ax.set_ylabel('Input x₂', fontsize=12, fontweight='bold')
ax.set_zlabel('Network Output', fontsize=12, fontweight='bold')
ax.set_title('Piecewise Linear Surface from 2D Neural Network\n'
             'The surface is made of flat planar regions stitched together', 
             fontsize=14, fontweight='bold', pad=20)

# Add colorbar
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Set viewing angle
ax.view_init(elev=25, azim=45)

plt.tight_layout()
plt.show()

print("Look closely at the surface - you can see the flat planar regions!")
print("Each flat region corresponds to a different activation pattern of the 4 hidden neurons.")

### Contour Plot: Seeing the Linear Regions

A contour plot makes the linear regions even more visible:

In [ ]:
# Create contour plot
fig, ax = plt.subplots(figsize=(12, 10))

# Filled contours
contourf = ax.contourf(X1, X2, Y, levels=30, cmap='viridis', alpha=0.8)

# Contour lines
contour = ax.contour(X1, X2, Y, levels=15, colors='white', 
                     linewidths=1, alpha=0.6)
ax.clabel(contour, inline=True, fontsize=8, fmt='%.2f')

# Labels and title
ax.set_xlabel('Input x₁', fontsize=13, fontweight='bold')
ax.set_ylabel('Input x₂', fontsize=13, fontweight='bold')
ax.set_title('Contour Plot: Linear Regions Are Visible\n'
             'Straight contour lines indicate linear regions', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_aspect('equal')

# Colorbar
cbar = plt.colorbar(contourf, ax=ax)
cbar.set_label('Network Output', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("Notice how the contour lines are STRAIGHT within each region!")
print("This is a telltale sign of piecewise linearity.")

### Cross-Section Analysis

Let's take a slice through the surface to see the piecewise linearity more clearly:

In [ ]:
# Take cross-sections at different x2 values
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

x2_values = [-1.5, -0.5, 0.5, 1.5]

for i, x2_fixed in enumerate(x2_values):
    # Find the index closest to x2_fixed
    idx = np.argmin(np.abs(x2_range - x2_fixed))
    cross_section = Y[idx, :]
    
    # Plot
    axes[i].plot(x1_range, cross_section, linewidth=2.5, color='darkblue')
    axes[i].set_xlabel('Input x₁', fontsize=11)
    axes[i].set_ylabel('Network Output', fontsize=11)
    axes[i].set_title(f'Cross-section at x₂ = {x2_fixed:.1f}', fontsize=12, fontweight='bold')
    axes[i].grid(True, alpha=0.3)
    
    # Add note
    axes[i].text(0.05, 0.95, 'Piecewise linear!', 
                transform=axes[i].transAxes, 
                fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))

plt.suptitle('Cross-Sections Through the 2D Surface\nEach slice is clearly piecewise linear', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("Every cross-section through the surface is piecewise linear!")
print("This holds true regardless of which direction we slice.")

---

## Part 3: Numerical Verification

Let's verify piecewise linearity numerically by checking that the second derivative is zero within regions.

In [ ]:
def check_linearity_along_line(start, end, W1, b1, W2, b2, n_points=100):
    """
    Check if the network function is linear along a line segment.
    Returns the maximum second difference (should be ~0 for linear segments).
    """
    # Generate points along the line
    t = np.linspace(0, 1, n_points)[:, np.newaxis]
    points = start + t * (end - start)
    
    # Evaluate network
    outputs = neural_network_2d(points, W1, b1, W2, b2).ravel()
    
    # Compute second differences (discrete approximation of second derivative)
    first_diff = np.diff(outputs)
    second_diff = np.diff(first_diff)
    
    max_second_diff = np.max(np.abs(second_diff))
    return max_second_diff

# Test multiple random line segments
print("="*70)
print("NUMERICAL VERIFICATION OF PIECEWISE LINEARITY")
print("="*70)
print("\nTesting linearity along random line segments:")
print("(If max second difference ≈ 0, the function is linear in that region)\n")

np.random.seed(123)
n_tests = 10
linear_count = 0

for i in range(n_tests):
    start = np.random.randn(2) * 2
    end = np.random.randn(2) * 2
    max_sd = check_linearity_along_line(start, end, W1, b1, W2, b2)
    
    is_linear = max_sd < 1e-8
    if is_linear:
        linear_count += 1
    
    status = "✓ LINEAR" if is_linear else "✗ Contains boundaries"
    print(f"Test {i+1:2d}: max |d²y| = {max_sd:.2e}  {status}")

print(f"\nResults: {linear_count}/{n_tests} line segments were perfectly linear")
print(f"         {n_tests - linear_count}/{n_tests} segments crossed activation boundaries")
print("\n✓ This confirms the piecewise linear structure!")

---

## Part 4: Activation Pattern Analysis

Each linear region corresponds to a unique activation pattern - which neurons are "on" (active) vs "off" (inactive).

In [ ]:
print("="*70)
print("ACTIVATION PATTERN ANALYSIS")
print("="*70)
print(f"\nNetwork has {W1.shape[1]} hidden neurons")
print(f"Maximum possible activation patterns: 2^{W1.shape[1]} = {2**W1.shape[1]}")
print(f"Each pattern corresponds to a different linear region\n")

# Sample points from different regions
sample_points = np.array([
    [-2.0, -2.0],
    [-1.0, -1.0],
    [0.0, 0.0],
    [1.0, 1.0],
    [2.0, 2.0],
    [-1.5, 1.5],
    [1.5, -1.5]
])

print("Sample points and their activation patterns:")
print("-" * 70)
print(f"{'Point':>15} | {'Pattern':>12} | {'Output':>8}")
print("-" * 70)

for point in sample_points:
    # Compute hidden layer activations
    hidden = np.maximum(0, point @ W1 + b1)
    
    # Binary activation pattern (1 = active, 0 = inactive)
    pattern = (hidden > 1e-10).astype(int)
    pattern_str = ''.join(map(str, pattern))
    
    # Network output
    output = neural_network_2d(point.reshape(1, -1), W1, b1, W2, b2)[0, 0]
    
    print(f"({point[0]:>5.1f}, {point[1]:>5.1f}) | {pattern_str:>12} | {output:>8.3f}")

print("-" * 70)
print("\nKey insight: Points with the same activation pattern are in the same")
print("             linear region and follow the same linear equation!")

---

## Part 5: Interactive Exploration (Optional)

Try modifying the network parameters and see how the piecewise linear structure changes!

In [ ]:
# Create a new network with different random weights
np.random.seed(999)  # Try different seeds: 42, 123, 999, etc.

W1_new = np.random.randn(2, 6) * 0.5  # Try more neurons!
b1_new = np.random.randn(6) * 0.5
W2_new = np.random.randn(6, 1) * 0.5
b2_new = np.random.randn(1) * 0.5

# Compute new surface
Y_new_flat = neural_network_2d(X_flat, W1_new, b1_new, W2_new, b2_new)
Y_new = Y_new_flat.reshape(X1.shape)

# Plot
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(X1, X2, Y_new, cmap=cm.plasma, 
                       alpha=0.9, linewidth=0, antialiased=True)

ax.set_xlabel('Input x₁', fontsize=12, fontweight='bold')
ax.set_ylabel('Input x₂', fontsize=12, fontweight='bold')
ax.set_zlabel('Network Output', fontsize=12, fontweight='bold')
ax.set_title(f'Different Network with {W1_new.shape[1]} Hidden Neurons\n'
             f'Still piecewise linear, but with more regions!', 
             fontsize=14, fontweight='bold', pad=20)

fig.colorbar(surf, ax=ax, shrink=0.5)
ax.view_init(elev=25, azim=45)

plt.tight_layout()
plt.show()

print(f"✓ This network has {W1_new.shape[1]} hidden neurons")
print(f"  Maximum possible regions: 2^{W1_new.shape[1]} = {2**W1_new.shape[1]}")
print("  More neurons = more linear regions = more complex function!")

---

## Summary and Key Takeaways

### What We've Learned

1. **ReLU networks are piecewise linear**: The entire function is composed of linear pieces stitched together

2. **Activation boundaries create regions**: Each neuron's activation threshold creates a boundary between linear regions

3. **Activation patterns define regions**: Each unique combination of active/inactive neurons corresponds to one linear region

4. **More neurons = more regions**: Networks with n neurons can have up to 2^n linear regions

5. **Universal approximation**: Piecewise linear functions can approximate any continuous function with enough pieces

### Why This Matters

- **Interpretability**: Understanding which linear region an input falls into helps explain predictions
- **Optimization**: The piecewise linear structure affects the optimization landscape
- **Adversarial robustness**: Boundaries between regions are potential points of vulnerability
- **Model compression**: Regions with similar linear equations can potentially be merged
- **Theoretical understanding**: Connects deep learning to classical approximation theory

### Going Deeper

This property extends to:
- Deeper networks (still piecewise linear!)
- Other activation functions (Leaky ReLU, PReLU, Maxout)
- Multi-output networks (each output is piecewise linear)
- Modern architectures like ResNets (when using ReLU)

**The piecewise linear structure is a fundamental geometric property of modern deep learning!**

---

## Further Exploration

Try experimenting with:
- Different numbers of neurons
- Different random seeds
- Deeper networks (add more layers)
- Higher dimensional inputs
- Different activation functions (e.g., Leaky ReLU)

See the accompanying Python scripts for more examples and the OVERVIEW.md for theoretical details.